# Generating preliminary results for the final aggregation classifier

In [2]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/ihranicky/git/domainradar-clf


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [3]:
create_test_parquet = True

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    input_files = [
        {'file': 'testdata/phishing_4.parquet', 'label': 'phishing', 'max_rows': 50000},
        {'file': 'testdata/benign_2312.parquet', 'label': 'benign', 'max_rows': 150000},
        {'file': 'testdata/umbrella_benign_FINISHED.parquet', 'label': 'benign', 'max_rows': 150000},
        {'file': 'testdata/malware_3.parquet', 'label': 'malware', 'max_rows': 50000},
        {'file': 'testdata/lex-dga-830k-pick.parquet', 'label': 'dga', 'max_rows': 50000},
    ]
    
    input_files = [
        {'file': 'testdata/phishing_4.parquet', 'label': 'phishing', 'max_rows': 50000},
        {'file': 'testdata/benign_2312.parquet', 'label': 'benign', 'max_rows': 100000},
        {'file': 'testdata/umbrella_benign_FINISHED.parquet', 'label': 'benign', 'max_rows': 100000},
        {'file': 'testdata/malware_3.parquet', 'label': 'malware', 'max_rows': 50000},
        {'file': 'testdata/lex-dga-830k-pick.parquet', 'label': 'dga', 'max_rows': 50000},
    ]

    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file']).head(first_file_info['max_rows'])
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file']).head(file_info['max_rows'])
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = -1
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)


    ROW_LIMIT = 100000 # (0 - no limit)

    # Subsample the combined dataframe to take N domains
    combined_df = combined_df.sample(n=ROW_LIMIT, random_state=42)
    
    # Shuffling the DataFrame
    shuffled_df = combined_df.sample(frac=1)

    # Save the selected rows to a new Parquet file
    shuffled_df.to_parquet('testdata/decision_training_xy.parquet')


/tmp/ipykernel_2105350/2919119430.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = -1
/tmp/ipykernel_2105350/2919119430.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = -1
/tmp/ipykernel_2105350/2919119430.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = -1
/

ArrowInvalid: ("Could not convert Timedelta('365 days 00:00:00') with type Timedelta: tried to convert to int64", 'Conversion failed for column rdap_registration_period with type object')

In [ ]:
shuffled_df

In [4]:
df.dtypes

domain_name                        object
label                              object
dns_has_dnskey                      int64
dns_A_count                         int64
dns_AAAA_count                      int64
                                    ...  
rdap_ip_longest_v6_prefix_len       int64
rdap_ip_avg_admin_name_len          int64
rdap_ip_avg_admin_name_entropy      int64
rdap_ip_avg_admin_email_len         int64
rdap_ip_avg_admin_email_entropy     int64
Length: 178, dtype: object

## Optional: Generate preliminary results for training the final aggregation classifier

In [5]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet('testdata/decision_training_xy.parquet')


# Time measurement start
import time
start_time = time.time()

# DO
preliminary_results_df = clf.generate_preliminary_results(input_df, add_final=False, output_file="preliminary_results_2406_a.parquet")

# Time measurement stop
end_time = time.time()
elapsed_time_seconds = end_time - start_time
elapsed_time_minutes = elapsed_time_seconds / 60
elapsed_time_hours = elapsed_time_seconds / 3600
print(f"Elapsed time: {elapsed_time_seconds:.2f} seconds")
print(f"Elapsed time: {elapsed_time_minutes:.2f} minutes")
print(f"Elapsed time: {elapsed_time_hours:.2f} hours")

2024-07-03 15:10:34.557981: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 15:10:34.558005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 15:10:34.558715: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-03 15:10:34.563700: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 15:10:35.162480: W tensorflow/compiler/tf2

CNN model created
Elapsed time: 50.59 seconds
Elapsed time: 0.84 minutes
Elapsed time: 0.01 hours


In [6]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,phishing_cnn_result,phishing_lgbm_result,phishing_xgboost_result,phishing_deepnn_result,phishing_dns_nn_result,phishing_rdap_nn_result,phishing_geo_nn_result,phishing_ip_nn_result,malware_lgbm_result,malware_xgboost_result,malware_deepnn_result,malware_dns_nn_result,malware_rdap_nn_result,malware_geo_nn_result,malware_ip_nn_result,dga_binary_deepnn_result,dga_binary_lgbm_result,phishing_sum,phishing_avg,phishing_prod,malware_sum,malware_avg,malware_prod,dga_binary_sum,dga_binary_avg,dga_binary_prod,total_sum,total_avg,total_prod,label
257696,tnet.ug,0.600000,0.125000,0.041667,0.000000,0.500000,0.000000,0.958333,0.166667,1.000000,0.000000,0.006230,0.001401,0.000806,0.002658,0.297143,0.072017,0.387495,0.414531,0.003933,0.006763,0.002359,0.661615,0.016481,0.604780,0.444184,0.065735,0.000773,1.182281,0.147785,0.000000,1.740115,0.248588,0.000000,0.066508,0.033254,0.000051,2.988904,0.175818,0.000000,malware
268211,asertinofase.top,0.600000,0.125000,0.041667,0.000000,0.500000,0.000000,0.833333,0.000000,1.000000,0.000000,0.549372,0.193137,0.458592,0.066232,0.650564,0.020158,0.387495,0.414531,0.592928,0.851123,0.012320,0.892006,0.024412,0.604780,0.444184,0.021205,0.000156,2.740082,0.342510,0.000007,3.421754,0.488822,0.000036,0.021360,0.010680,0.000003,6.183196,0.363717,0.000000,malware
119315,rvnews.rv.ua,1.000000,0.700000,0.833333,0.250000,1.000000,1.000000,1.000000,0.708333,1.000000,1.000000,0.000599,0.000162,0.000413,0.000099,0.007011,0.001388,0.009992,0.018285,0.000051,0.002233,0.000000,0.037618,0.000019,0.000730,0.000062,0.000026,0.000003,0.037949,0.004744,0.000000,0.040714,0.005816,0.000000,0.000029,0.000015,0.000000,0.078692,0.004629,0.000000,benign
42200,postcts.top,0.900000,0.525000,0.041667,0.000000,1.000000,1.000000,0.833333,0.416667,1.000000,1.000000,0.232186,0.021882,0.591706,0.942014,0.678680,0.421857,0.937632,0.969719,0.000093,0.002233,0.000000,0.561788,0.000002,0.002864,0.000910,0.007233,0.000360,4.795677,0.599460,0.000737,0.567891,0.081127,0.000000,0.007593,0.003796,0.000003,5.371161,0.315951,0.000000,phishing
114073,bn.gov.br,1.000000,0.725000,1.000000,0.708333,1.000000,0.875000,0.833333,0.291667,1.000000,1.000000,0.000154,0.000173,0.000397,0.000145,0.003111,0.001767,0.063655,0.027072,0.000755,0.003614,0.000000,0.016928,0.000208,0.213101,0.000145,0.001224,0.000011,0.096475,0.012059,0.000000,0.234751,0.033536,0.000000,0.001234,0.000617,0.000000,0.332460,0.019556,0.000000,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35153,lp-mypwd.pages.dev,0.900000,0.575000,1.000000,0.833333,1.000000,0.875000,1.000000,0.833333,1.000000,1.000000,0.992934,0.712206,0.654344,0.999985,0.954295,0.917266,0.134651,0.527485,0.000072,0.002233,0.000000,0.789398,0.000035,0.000013,0.000059,0.000000,0.000001,5.893166,0.736646,0.028769,0.791812,0.113116,0.000000,0.000001,0.000001,0.000000,6.684979,0.393234,0.000000,phishing
318330,upfsdsnwlvkwrc.org,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.041667,0.000000,0.000000,1.000000,0.001820,0.002121,0.000000,0.065006,0.000001,0.169900,0.145029,0.022390,0.149198,0.000000,0.573180,0.000000,0.383097,0.000000,0.999986,0.999770,1.383877,0.172985,0.000000,1.127865,0.161124,0.000000,1.999756,0.999878,0.999756,4.511498,0.265382,0.000000,dga
149800,moodle.kramuo.dn.ua,0.600000,0.225000,0.833333,0.291667,1.000000,0.625000,1.000000,0.708333,1.000000,0.722222,0.103472,0.000349,0.000520,0.004185,0.015606,0.024849,0.029801,0.444768,0.001416,0.002711,0.000138,0.021669,0.048151,0.172861,0.346479,0.000000,0.000002,0.623549,0.077944,0.000000,0.593425,0.084775,0.000000,0.000002,0.000001,0.000000,1.216975,0.071587,0.000000,benign
230410,fastly.newssuite.sinfony.ne.jp,0.900000,0.525000,0.833333,0.250000,1.000000,0.375000,0.833333,0.125000

In [7]:
preliminary_results_df[["domain_name", "label", "phishing_cnn_result", "phishing_lgbm_result", "malware_cnn_result", "malware_xgboost_result", "dga_binary_nn_result"]]

KeyError: "['malware_cnn_result', 'dga_binary_nn_result'] not in index"